<a href="https://colab.research.google.com/github/benchov/Machine_Learning_for_Trading_Knowledge/blob/main/PCA_and_Dimensionality_Reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ta
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29106 sha256=f47d6ba358cbf60951a0e86fdc57911c6b9919d65379773de31c61b3a81a4d65
  Stored in directory: /root/.cache/pip/wheels/1b/81/50/db21d9643326c61dd6fee1dc2562d8351ee7753164c24ead38
Successfully built ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.2 MB 4.3 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109 kB 4.3 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully unin

In [14]:
# Remove unwanted warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

# Data Management
import pandas as pd
import numpy as np
from pandas_datareader.data import DataReader
from ta import add_all_ta_features
import yfinance as yf

# Statistics
from statsmodels.tsa.stattools import adfuller

# Unsupervised Machine Learning
from sklearn.decomposition import PCA

# Supervised Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt

In [18]:
# Data Extraction|
start_date = '2017-01-01'
end_date = '2022-06-01'
symbol = "^VIX"
df = yf.download(symbol, start=start_date, end=end_date)
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,14.07,14.07,12.85,12.85,12.85,0
2017-01-04,12.78,12.80,11.63,11.85,11.85,0
2017-01-05,11.96,12.09,11.40,11.67,11.67,0
2017-01-06,11.70,11.74,10.98,11.32,11.32,0
2017-01-09,11.71,12.08,11.46,11.56,11.56,0


In [19]:
# add TA features
df = add_all_ta_features(df, open="Open", high='High', low="Low", close='Adj Close', volume="Volume", fillna=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1362 entries, 2017-01-03 to 2022-05-31
Data columns (total 92 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Open                       1362 non-null   float64
 1   High                       1362 non-null   float64
 2   Low                        1362 non-null   float64
 3   Close                      1362 non-null   float64
 4   Adj Close                  1362 non-null   float64
 5   Volume                     1362 non-null   int64  
 6   volume_adi                 1362 non-null   float64
 7   volume_obv                 1362 non-null   int64  
 8   volume_cmf                 1362 non-null   float64
 9   volume_fi                  1362 non-null   float64
 10  volume_em                  1362 non-null   float64
 11  volume_sma_em              1362 non-null   float64
 12  volume_vpt                 1362 non-null   float64
 13  volume_vwap                136

In [21]:
# data preprocessing - stationarity
non_stationaries = []
for col in df.columns:
  dftest = adfuller(df[col].values)
  p_value = dftest[1]
  t_test = dftest[0] < dftest[4]['1%']
  if p_value > 0.05 or not t_test:
    non_stationaries.append(col)
print(f"Non-Stationary Features Found: {len(non_stationaries)}")

Non-Stationary Features Found: 29
